In [21]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [22]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where r_comments like 'Verified%' and product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

In [23]:
df = retrieve_reviews('Samsung Electronics UN65MU6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')
columns = ['id','product','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']
df.columns = columns
df.head(2)

,id,product,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,475,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,1,5.0 out of 5 stars,"August 28, 2017",James Barrow,Someone here complained that the television is...,"Before I purchased this television, I read alm...",Verified Purchase Helpful No...
1,477,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,3,5.0 out of 5 stars,"November 29, 2017",Y. L.,Awesome TV-DONT ACCEPT ADS AGREEMENT,WATCH OUT FOR the TV ADS AGREEMENT DURING SET ...,Verified Purchase Helpful


In [24]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new['reviews'] = df['r_title'] + ' ' + df['r_text']
df_new = df_new[['rating','product','reviews']]
df_new.head(3)

,rating,product,reviews
0,1,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,Someone here complained that the television is...
1,1,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,Awesome TV-DONT ACCEPT ADS AGREEMENT WATCH OUT...
2,0,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,Cant use Netflix app unless you agree to allow...


In [25]:
from src.LdaReviewAnalyzer import LdaReviewAnalyzer

In [26]:
lda = LdaReviewAnalyzer(df_new)
lda.fit_transform()
lda.print_topics()

Topic 0:
[('samsung', 238.5661646834203), ('screen', 105.14104445329636), ('remote', 76.97705230606127), ('box', 62.105416497502446), ('just', 58.66837124429656), ('service', 56.662115355258635), ('work', 53.74027367079558), ('picture', 53.650501187639904), ('working', 49.768236775585954), ('time', 49.62918877940393)]
Topic 1:
[('hdr', 40.40845130937271), ('problems', 25.56939853816092), ('television', 17.326888196281917), ('input', 16.1319596723529), ('stars', 15.741996703842064), ('want', 15.120340028322659), ('settings', 14.881456957565002), ('hooked', 14.486182689427416), ('like', 14.162318573234211), ('viewing', 13.647589627359094)]
Topic 2:
[('love', 87.03896018050305), ('picture', 77.08447305998313), ('remote', 71.5856739763778), ('awesome', 62.45063484491005), ('easy', 61.06557258834266), ('happy', 51.74094115505423), ('television', 48.9438865813145), ('set', 41.67445824285048), ('control', 40.50030029251603), ('great', 40.15300047599635)]
Topic 3:
[('great', 622.7584482380134)

In [27]:
lda.display_topic_model()